In [1]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

Missouri
--------
Election results from the Secretary of State's office via OpenElections (https://github.com/openelections/openelections-data-mo/tree/master/2016). Absentee,provisional, and mail ballots were reported countywide in nearly every county; these were distributed by candidate to precincts based on their share of the precinct-level reported vote.

Precinct shapefiles primarily from the U.S. Census Bureau's 2020 Redistricting Data Program Phase 2 release. Monroe County comes from the 2010 Census VTD release. Platte County comes from the 2020 Census VTD release.  The following counties instead used maps or shapefiles sourced from the respective county governments: Audrain, Bates, Caldwell, Callaway, Camden, Cape Girardeau, Cedar, Cooper, Franklin, Greene, Jackson, Jasper, Jefferson, Laclede, Lafayette, Lawrence, Marion, McDonald, Miller, Nodaway, Osage, Ozark, Pemiscot, Pike, Randolph, Scott, Ste. Genevieve, Texas, Warren, Washington, Worth, Wright.

Precinct mergers were made in the following counties to match county reporting units: Andrew, Audrain, Barry, Benton, Bollinger, Boone, Callaway, Camden, Cape Girardeau, Carroll, Carter, Cass, Christian, Clark, Cole, Cooper, Crawford, Dallas, Dekalb, Douglas, Dunklin, Franklin, Gentry, Greene, Grundy, Howard, Howell, Jackson, Jasper, Jefferson, Johnson, Linn, Livingston, Marion, Mercer, Mississippi, Moniteau, Morgan, New Madrid, Nodaway, Oregon, Osage, Pemiscot, Pettis, Phelps, Platte, Polk, Putnam, Ralls, Randolph, Ray, Ripley, St. Clair, St. Louis, Schuyler, Shannon, Shelby, Stone, Sullivan, Warren, Wayne, Webster, Worth.

Additional modifications to reflect how election results were reported in 2016:

Audrain: Add Vandalia City; Align Mexico precincts with municipal code
Barry: Add Monett City; Align Cassville with corporate boundary
Barton: Adjust Lamar ward boundaries to match city map
Bollinger: Adjust Patton/Sedgewickville to match county GIS
Butler: Align Poplar Bluff with corporate boundary and adjust wards to match city map
Callaway: Align Fulton, Holts Summit precincts with 2016 corporate boundaries
Carroll: Adjust Carrollton City wards to match voter file
Cass: Align Belton, Pleasant Hill, Raymore precincts with corporate boundaries; Adjust 37 Pleasant Prairie/25 West Peculiar Rural
Chariton: Adjust Salisbury ward boundaries to match city map
Christian: Revise Cassidy 1/Cassidy 2; Reverse Nixa City annexations into Rosedale 1A/1B, Union Chapel 1
Clay: Adjust Excelsior Springs, Kearney, Smithville precincts to match 2016 ward and corporate boundaries; Adjust KC 21-1/15, 21-1/17, 21-3/11, 21-4/6, 21-4/11, 21-5/7/24, 21-6/24, 21-7/20, 21-13/26, 21-14/23, 21-17/16, 21-17/26, 21-17/Platte 1, Gallatin 7/16, 10/13, 10/16, 11/15, 13/15, Kearney 3/Platte 2, Liberty 1/10, 6/13, 7/11, 11/14, Washington 1/3 to match voter file
Clinton: Align Atchison/Concord, Jackson/Lathrop with 2016 precinct split assignments
Cole: Adjust Marion Twp St Martins to include St Martins city in Jefferson Twp
Crawford: Move Benton Twp section of Leasburg precinct from Recklein to Leasburg
Dade: Realign Greenfield wards to match municipal code
Dallas: Split North Benton Inside/OCL; Align N/S Benton with Buffalo ward map
Dent: Split Boss precinct from R-4 precinct; Adjust R-2/R-3 boundary
Dunklin: Align Holcomb, Rives with township boundaries; Adjust Arbyrd/Senath; Align Kennett wards with city map
Franklin: Split Dry Branch between Stanton & St. Clair Out of Town
Gasconade: Align Hermann, Owensville, Rosebud with corporate boundaries and ward descriptions
Howard: Split Old Franklin & New Franklin
Howell: Align West Plains precincts with 2016 corporate boundary
Iron: Adjust Annapolis/Des Arc/Vulcan, Ironton/Pilot Knob
Jasper: Realign Carthage 2/3, Joplin 1/7, 10/13, 10/15, 13/22, Oronogo 1/2, Webb City 1/4 to match city ward maps
Jefferson: Split Festus Ward 4 P1/P2; Adjust Crystal City/Riverview
Johnson: Adjust Warrensburg NE/NW boundary to match county maps
Lewis: Split LaBelle Twp between LaBelle FS/Lewistown FS with PLSS grid
Lincoln: Align Moscow Mills, Troy with 2016 corporate boundaries; Adjust Briscoe/Troy Rural
Linn: Split Brookfield Township between Brookfield 1&2/3&4
Livingston: Align Chillicothe with 2016 corporate boundaries
Macon: Align Macon City wards with corporate boundaries
Madison: Align Fredericktown wards with corporate boundaries
Marion: Split Fabius-Smileyville/Taylor, Warren-Monroe City/Warren; Align Hannibal wards with voter file
Miller: Adjust all precinct boundaries to match voter file
Mississippi: Adjust Bertrand/East Prairie Rural; Align East Prairie City with corporate boundary
Montgomery: Align Bellflower/Middletown, Big Spring/Rhineland with voter file
Morgan: Adjust Versaille North/South boundary to match county GIS
New Madrid: Add New Madrid City wards; Add Big Prairie 4 precinct; Move Portageville Ward 1 to Portage 1
Newton: Align Staples precincts with Joplin City GIS boundaries; Align Neosho precincts with Neosho City GIS boundaries; Adjust Newtonia/Ritchey Wentworth
Pettis: Adjust Sedalia City wards to match city map
Phelps: Align St. James ward boundaries with city map
Pike: Adjust Cuivre JL boundaries with Ashley NO, Buffalo E, Prairieville I
Platte: Align Lake Waukomis, Parkville, Platte City, Riverside/Northmore, Weatherby Lake with corporate boundaries; Adjust Platte Hills/Park Hill, Seven Bridges/Shiloh
Polk: Split Campbell/Jefferson precincts by school district
Pulaski: Adjust Big Piney/St. Roberts, Laquey/Waynesville
Putnam: Split Union Twp into NE/SW precincts to match county map
Ralls: Replace Center/Liberty precincts with 2010 VTDs
Ray: Adjust Richmond City wards, Homestead Village/Wood Heights/Lawson
Reynolds: Realign precincts countywide to match voter file
Saline: Split Nelson precinct from Hardeman precinct; Adjust Marshall Ward 2/3 boundary
St. Charles: Adjust Brian/Ridgepoint, Civic/St. Paul, Harvester/Sycamore, Mamelle/Marina
St. Francois: Align Desloge, Farmington, Park Hills with corporate boundaries and city ward maps; Adjust Bonne Terre/Timberline, Patterson/Timberline, Leadwood/Park Hills to match voter file
St. Louis: Add UNV044 precinct; Revise QUE047 precinct
Scott: Adjust Haywood City/Morley, Sikeston Ward 1/McMullin/Vanduser
Shannon: Align all precincts with county GIS and voter file
Shelby: Adjust Jackson/North River to match voter file
Stoddard: Align Advance, Bernie, Bloomfield, Dexter, Puxico with corporate boundaries; Adjust Advance/Leora, Bernie/Dexter/Pyle, Bloomfield/Gray Ridge
Stone: Move Branson West from Ruth A to Ruth C; Adjust Ruth B City/Rural to match county GIS
Taney: Adjust Boston Center/Walnut Shade; Align Branson wards with 2016 corporate boundary
Vernon: Align Bronaugh/Moundville, Harwood/Schell City, Deerfield/NG Armory with voter file
Washington: Adjust City Hall/Stony Point, Cruise/Richwoods to match county GIS



In [19]:
vest_mo_16 = gp.read_file("./raw-from-source/VEST/mo_2016/mo_2016.shp")

In [2]:
fips_file = pd.read_csv("./raw-from-source/FIPS/US_FIPS_Codes.csv")
fips_file = fips_file[fips_file["State"]=="Missouri"]
fips_file["FIPS County"]=fips_file["FIPS County"].astype(str)
fips_file["FIPS County"]=fips_file["FIPS County"].str.zfill(3)
fips_file["unique_ID"] =  "42" + fips_file["FIPS County"]
fips_codes = fips_file["unique_ID"].tolist()
mo_fips_dict = dict(zip(fips_file["County Name"],fips_file["FIPS County"]))

In [3]:
elections_2016 = pd.read_csv("./raw-from-source/Open_Elections/openelections-data-mo-master/2016/20161108__mo__general__precinct.csv")

In [4]:
office_list = ['President', 'U.S. Senate', 'Governor', 'Lieutenant Governor','Secretary of State', 'State Treasurer', 'Attorney General']
elections_2016 = elections_2016[elections_2016["office"].isin(office_list)]

G16PRERTRU - Donald J. Trump (Republican Party)
G16PREDCLI - Hillary Rodham Clinton (Democratic Party)
G16PRELJOH - Gary Johnson (Libertarian Party)
G16PREGSTE - Jill Stein (Green Party)
G16PRECCAS - Darrell L. Castle (Constitution Party)

G16USSRBLU - Roy Blunt (Republican Party)
G16USSDKAN - Jason Kander (Democratic Party)
G16USSLDIN - Jonathan Dine (Libertarian Party)
G16USSGMCF - Johnathan McFarland (Green Party)
G16USSCRYM - Fred Ryman (Constitution Party)

G16GOVRGRE - Eric Greitens (Republican Party)
G16GOVDKOS - Chris Koster (Democratic Party)
G16GOVLSPR - Cisse W. Spragins (Libertarian Party)
G16GOVGFIT - Don Fitz (Green Party)
G16GOVITUR - Lester Benton (Les) Turilli, Jr. (Independent)

G16LTGRPAR - Mike Parson (Republican Party)
G16LTGDCAR - Russ Carnahan (Democratic Party)
G16LTGLHED - Steven R. Hedrick (Libertarian Party)
G16LTGGLEA - Jennifer Leach (Green Party)

G16ATGRHAW - Josh Hawley (Republican Party)
G16ATGDHEN - Teresa Hensley (Democratic Party)

G16TRERSCH - Eric Schmitt (Republican Party)
G16TREDBAK - Judy Baker (Democratic Party)
G16TRELOTO - Sean O'Toole (Libertarian Party)
G16TREGHEX - Carol Hexem (Green Party)

G16SOSRASH - John (Jay) Ashcroft (Republican Party)
G16SOSDSMI - Robin Smith (Democratic Party)
G16SOSLMOR - Chris Morrill (Libertarian Party)

In [5]:
print(elections_2016["candidate"].value_counts())

Donald J. Trump, Michael R. Pence                   3228
Roy Blunt                                           3225
Josh Hawley                                         3222
Eric Schmitt                                        3222
John (Jay) Ashcroft                                 3222
Eric Greitens                                       3221
Mike Parson                                         3221
Hillary Rodham Clinton, Timothy Michael Kaine       3217
Jason Kander                                        3214
Chris Koster                                        3213
Teresa Hensley                                      3210
Robin Smith                                         3209
Russ Carnahan                                       3209
Judy Baker                                          3209
Chris Morrill                                       3191
Gary Johnson, Bill Weld                             3186
Steven R. Hedrick                                   3185
Jonathan Dine                  

In [6]:
candidate_list = ["Donald J. Trump, Michael R. Pence","Roy Blunt","Eric Schmitt","Josh Hawley",
                 "John (Jay) Ashcroft","Eric Greitens","Mike Parson","Hillary Rodham Clinton, Timothy Michael Kaine",
                 "Jason Kander","Chris Koster","Teresa Hensley","Judy Baker","Russ Carnahan","Robin Smith","Chris Morrill",
                 "Gary Johnson, Bill Weld","Jonathan Dine","Steven R. Hedrick","Sean O'Toole","Jennifer Leach","Fred Ryman",
                 "Cisse W Spragins","Johnathan McFarland","Carol Hexem","Lester Benton (Les) Turilli, Jr.","Don Fitz",
                 "Jill Stein, Ajamu Baraka","Darrell L. Castle, Scott N. Bradley"]

print(len(candidate_list))

elections_2016=elections_2016[elections_2016["candidate"].isin(candidate_list)]

28


In [7]:
print(elections_2016["candidate"].value_counts())
print(len(elections_2016["candidate"].value_counts()))

Donald J. Trump, Michael R. Pence                3228
Roy Blunt                                        3225
Josh Hawley                                      3222
John (Jay) Ashcroft                              3222
Eric Schmitt                                     3222
Mike Parson                                      3221
Eric Greitens                                    3221
Hillary Rodham Clinton, Timothy Michael Kaine    3217
Jason Kander                                     3214
Chris Koster                                     3213
Teresa Hensley                                   3210
Robin Smith                                      3209
Russ Carnahan                                    3209
Judy Baker                                       3209
Chris Morrill                                    3191
Gary Johnson, Bill Weld                          3186
Jonathan Dine                                    3185
Steven R. Hedrick                                3185
Sean O'Toole                

In [8]:
elections_2016["county"]=elections_2016["county"].map(mo_fips_dict).fillna(elections_2016["county"])

In [9]:
print(elections_2016.head())
elections_2016["pivot_col"]=elections_2016["county"]+elections_2016["precinct"]

  county                precinct     office district party  \
0    001    SOUTHWEST ONE/BENTON  President      NaN   DEM   
1    001    SOUTHEAST TWO/BENTON  President      NaN   DEM   
2    001  SOUTHEAST THREE/BENTON  President      NaN   DEM   
3    001   NORTHEAST FOUR/BENTON  President      NaN   DEM   
4    001   NORTHEAST FIVE/BENTON  President      NaN   DEM   

                                       candidate  votes  
0  Hillary Rodham Clinton, Timothy Michael Kaine    224  
1  Hillary Rodham Clinton, Timothy Michael Kaine    458  
2  Hillary Rodham Clinton, Timothy Michael Kaine    391  
3  Hillary Rodham Clinton, Timothy Michael Kaine    386  
4  Hillary Rodham Clinton, Timothy Michael Kaine    256  


In [12]:
pivoted_2016 = pd.pivot_table(elections_2016,values=["votes"],index=["pivot_col"],columns=["candidate"],aggfunc=sum)
pivoted_2016 = pivoted_2016.fillna(0)

In [13]:
#print(pivoted_2016.head())
pivoted_2016.reset_index(drop=False,inplace=True)
pivoted_2016.columns = pivoted_2016.columns.droplevel(0)

print(pivoted_2016.head())

candidate                            Carol Hexem  Chris Koster  Chris Morrill  \
0                       001ABSENTEE         13.0         570.0           40.0   
1                       001BRASHEAR          4.0         245.0           28.0   
2                        001FEDERAL          0.0           0.0            0.0   
3          001NORTHEAST FIVE/BENTON         13.0         304.0           21.0   
4          001NORTHEAST FOUR/BENTON         20.0         427.0           42.0   

candidate  Cisse W Spragins  Darrell L. Castle, Scott N. Bradley  Don Fitz  \
0                      19.0                                  8.0       3.0   
1                       8.0                                  3.0      13.0   
2                       0.0                                  0.0       0.0   
3                       3.0                                  4.0       5.0   
4                      21.0                                  2.0      13.0   

candidate  Donald J. Trump, Michael R. Pence

In [14]:
print(pivoted_2016.columns)

Index(['', 'Carol Hexem', 'Chris Koster', 'Chris Morrill', 'Cisse W Spragins',
       'Darrell L. Castle, Scott N. Bradley', 'Don Fitz',
       'Donald J. Trump, Michael R. Pence', 'Eric Greitens', 'Eric Schmitt',
       'Fred Ryman', 'Gary Johnson, Bill Weld',
       'Hillary Rodham Clinton, Timothy Michael Kaine', 'Jason Kander',
       'Jennifer Leach', 'Jill Stein, Ajamu Baraka', 'John (Jay) Ashcroft',
       'Johnathan McFarland', 'Jonathan Dine', 'Josh Hawley', 'Judy Baker',
       'Lester Benton (Les) Turilli, Jr.', 'Mike Parson', 'Robin Smith',
       'Roy Blunt', 'Russ Carnahan', 'Sean O'Toole', 'Steven R. Hedrick',
       'Teresa Hensley'],
      dtype='object', name='candidate')


In [15]:
sum_var = 0
for i in ['Carol Hexem', 'Chris Koster', 'Chris Morrill', 'Cisse W Spragins',
       'Darrell L. Castle, Scott N. Bradley', 'Don Fitz',
       'Donald J. Trump, Michael R. Pence', 'Eric Greitens', 'Eric Schmitt',
       'Fred Ryman', 'Hillary Rodham Clinton, Timothy Michael Kaine',
       'Jason Kander', 'Jennifer Leach', 'Jill Stein, Ajamu Baraka',
       'John (Jay) Ashcroft', 'Johnathan McFarland', 'Jonathan Dine',
       'Josh Hawley', 'Judy Baker', 'Lester Benton (Les) Turilli, Jr.',
       'Mike Parson', 'Robin Smith', 'Roy Blunt', 'Russ Carnahan',
       "Sean O'Toole", 'Steven R. Hedrick', 'Teresa Hensley']:
    sum_var += np.sum(pivoted_2016[i])
print(sum_var)

19321091.0


In [20]:
sum_var = 0
for i in ['G16PRERTRU', 'G16PREDCLI', 'G16PRELJOH',
       'G16PREGSTE', 'G16PRECCAS', 'G16USSRBLU', 'G16USSDKAN', 'G16USSLDIN',
       'G16USSGMCF', 'G16USSCRYM', 'G16GOVRGRE', 'G16GOVDKOS', 'G16GOVLSPR',
       'G16GOVGFIT', 'G16GOVITUR', 'G16LTGRPAR', 'G16LTGDCAR', 'G16LTGLHED',
       'G16LTGGLEA', 'G16ATGRHAW', 'G16ATGDHEN', 'G16TRERSCH', 'G16TREDBAK',
       'G16TRELOTO', 'G16TREGHEX', 'G16SOSRASH', 'G16SOSDSMI', 'G16SOSLMOR']:
    sum_var += np.sum(vest_mo_16[i])
print(sum_var)

19418450


In [21]:
pivoted_2016 = pivoted_2016[['G16PRERTRU', 'G16PREDCLI', 'G16PRELJOH',
       'G16PREGSTE', 'G16PRECCAS', 'G16USSRBLU', 'G16USSDKAN', 'G16USSLDIN',
       'G16USSGMCF', 'G16USSCRYM', 'G16GOVRGRE', 'G16GOVDKOS', 'G16GOVLSPR',
       'G16GOVGFIT', 'G16GOVITUR', 'G16LTGRPAR', 'G16LTGDCAR', 'G16LTGLHED',
       'G16LTGGLEA', 'G16ATGRHAW', 'G16ATGDHEN', 'G16TRERSCH', 'G16TREDBAK',
       'G16TRELOTO', 'G16TREGHEX', 'G16SOSRASH', 'G16SOSDSMI', 'G16SOSLMOR']]

KeyError: "None of [Index(['G16PRERTRU', 'G16PREDCLI', 'G16PRELJOH', 'G16PREGSTE', 'G16PRECCAS',\n       'G16USSRBLU', 'G16USSDKAN', 'G16USSLDIN', 'G16USSGMCF', 'G16USSCRYM',\n       'G16GOVRGRE', 'G16GOVDKOS', 'G16GOVLSPR', 'G16GOVGFIT', 'G16GOVITUR',\n       'G16LTGRPAR', 'G16LTGDCAR', 'G16LTGLHED', 'G16LTGGLEA', 'G16ATGRHAW',\n       'G16ATGDHEN', 'G16TRERSCH', 'G16TREDBAK', 'G16TRELOTO', 'G16TREGHEX',\n       'G16SOSRASH', 'G16SOSDSMI', 'G16SOSLMOR'],\n      dtype='object', name='candidate')] are in the [columns]"